In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os.path
import os
from scipy.interpolate import interp1d
from scipy.signal import butter,filtfilt
from scipy.optimize import  curve_fit
from scipy import interpolate
from scipy import odr

%run python_master.py
%matplotlib qt 

# Imort frequency files

In [3]:
folder = "C:\\Users\\Florian\\Box Sync\\Projects"
project = "\\Mn3X\\Mn3.1Sn0.89\\RUS\\2010A\\good_data\\shift"
# project = "\\Mn3X\\Mn3.019Sn0.981\\RUS\\2007A\\good_data\\bad"

data_files = os.listdir(folder+project)
data_files = [i for i in data_files if i[-4:]=='.dat']
filenames = [folder + project + '\\' + i for i in data_files[:]]
data_files

['1923_no_outliers_cooling_210324_clean.dat',
 '2272_no_outliers_cooling_210305_clean.dat',
 '2273_no_outliers_cooling_210325_clean.dat']

In [4]:
T = []
f = []
g = []
Tint = np.linspace(427.5, 429.5, 170)
fint = []
gint = []
for file in filenames:
    data = data_extract(file)
    Ti = data.import_data(1,2)[0]
    fi = data.import_data(1,2)[1]
    # gi = data.import_data(1,3)[2]
 
    # g.append(gi[np.argsort(Ti)])
    f.append(fi[np.argsort(Ti)])
    T.append(np.sort(Ti))

    fint.append( interp1d(Ti, fi, kind='linear')(Tint) )
    # gint.append( interp1d(Ti, gi, kind='linear')(Tint) )

In [5]:
der = [np.gradient(F) for F in fint]
Tmin = 427.5
Tmax = 429.5
mask = [(t>Tmin)&(t<Tmax) for t in T]
for ii, t in enumerate(T):
    plt.plot( t[mask[ii]], (f[ii][mask[ii]]-max(f[ii][mask[ii]]))/(max(f[ii][mask[ii]])-min(f[ii][mask[ii]])) + ii )
    plt.plot( Tint, (der[ii]-max(der[ii]))/(max(der[ii])-min(der[ii])) + ii )

In [6]:
Tmin = 427.5
Tmax = 429.5
maskint = (Tint>Tmin)&(Tint<Tmax)

maxT = []
maskfit = []
# Tfit = []
for ii, _ in enumerate(T):
    # ti = t[mask[ii]]
    deri = der[ii][maskint]
    ti = Tint[maskint]
    dermax = max(deri)
    index = ( np.abs( deri - dermax ) == 0 )
    tmax = ti[index][0]
    maxT.append(tmax)
    fitmask = (Tint>tmax-0.3)&(Tint<tmax+0.3)
    maskfit.append( fitmask )
    # Tfit.append(np.linspace(tmax-0.3, tmax+0.3, 1000))

fit = []
degree = 2

maxTemps = []
for ii, t in enumerate(T):
    fitii = ( np.polyval( np.polyfit(Tint[maskfit[ii]], der[ii][maskfit[ii]], degree), Tint[maskfit[ii]] ) )
    fit.append(fitii)

    index = ( np.abs( fitii - max(fitii) ) == 0 )
    maxTemps.append(np.mean(Tint[maskfit[ii]][index]))


for ii, t in enumerate(T):
    plt.plot( t[mask[ii]], (f[ii][mask[ii]]-max(f[ii][mask[ii]]))/(max(f[ii][mask[ii]])-min(f[ii][mask[ii]])) + ii )
    plt.plot( Tint, (der[ii]-max(der[ii]))/(max(der[ii])-min(der[ii])) + ii )
    plt.plot ( Tint[maskfit[ii]], (fit[ii]-max(der[ii]))/(max(der[ii])-min(der[ii])) + ii )
    # plt.plot ( Tfit[ii], (fit[ii]-max(fit[ii]))/(max(fit[ii])-min(fit[ii])) + ii)

In [7]:
Tright = maxTemps[0]
Tshift = np.array(maxTemps) - Tright
T_shifted = [t-Tshift[ii] for ii,t in enumerate(T)]

In [8]:
for ii, t in enumerate(T_shifted):
    plt.plot( t[mask[ii]], (f[ii][mask[ii]]-max(f[ii][mask[ii]]))/(max(f[ii][mask[ii]])-min(f[ii][mask[ii]])) + ii )
    # plt.plot( t[mask[ii]], (der[ii][mask[ii]]-max(der[ii][mask[ii]]))/(max(der[ii][mask[ii]])-min(der[ii][mask[ii]])) + ii )

# save data

In [9]:
for ii, file in enumerate(filenames):
    save_filename = file[:-4] + '_shifted.dat'
    if os.path.isfile(save_filename) == True:
        x='w'
    else:
        x='x'
    
    with open(save_filename, x) as clean:
        clean.write('T (K)\tf (kHz)\twidth (kHz)\n ')
        for i in np.arange(len(f[ii])-1):
            # clean.write(str(T_shifted[ii][i]) + '\t' + str(f[ii][i]) + '\t' + str(g[ii][i]) + '\n')
            clean.write(str(T_shifted[ii][i]) + '\t' + str(f[ii][i]) + '\n')